In [1]:
#nltk.download('stopwords')
#nltk.download('punkt')
#!python -m spacy download es_core_news_md
#!python -m spacy validate

import warnings
import pandas as pd
warnings.filterwarnings("ignore", category=DeprecationWarning)
pd.options.mode.chained_assignment = None  # default='warn'

from tqdm import tqdm
from tqdm.notebook import tqdm_notebook
tqdm.pandas()
import numpy as np

from utils import clean_dataset_basedOn_media
from utils import cluster_by_month

In [2]:
df = pd.read_csv("../data/loslagos-comunas.csv")
df = cluster_by_month(clean_dataset_basedOn_media(df))
df.isna().any()

date               False
media_outlet       False
url                False
title              False
text               False
content            False
comuna              True
date_clustering    False
dtype: bool

In [3]:
df.date_clustering.value_counts()

2022-03    4617
2021-12    4492
2021-10    4370
2021-11    4326
2022-01    4182
2022-02    3950
2022-04    3384
Name: date_clustering, dtype: int64

In [ ]:
# Obtenemos las etiquetas del value_counts 
months = df.date_clustering.value_counts().index.tolist()

# se hará un análisis del primer mes
import datetime
dates = [datetime.datetime.strptime(ts, "%Y-%m") for ts in months]
dates.sort()
sorteddates = [datetime.datetime.strftime(ts, "%Y-%m") for ts in dates]

In [ ]:
##### Queremos seleccionar por comuna?
#selection = ['puerto montt']
#mask = df.comuna.apply(lambda x: any(item for item in selection if item in x))
#df1 = df[mask]

# [BERTopic](https://maartengr.github.io/BERTopic/index.html)
BERTopic is a topic modeling technique that leverages transformers and c-TF-IDF to create dense clusters allowing for easily interpretable topics whilst keeping important words in the topic descriptions.

In [ ]:
#!pip install bertopic
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
selected = df[df.date_clustering == sorteddates[0]]
docs = selected.content.tolist()
topic_model = BERTopic(language="spanish",
                       calculate_probabilities=True,
                       verbose=True,
                       nr_topics=15)

topics, probs = topic_model.fit_transform(docs)

In [ ]:
freq = topic_model.get_topic_info()
print(freq)

In [ ]:
fig = topic_model.visualize_topics()
fig.write_image("../img/fig0411.png")

<img src="../img/fig0411.png"/>

In [ ]:
labels = topic_model.generate_topic_labels()
print(labels)

In [ ]:
topic_model.get_topic(0)  # Select the most frequent topic

In [ ]:
fig = topic_model.visualize_barchart()
fig.write_image("../img/fig0412.png")

<img src="../img/fig0412.png"/>

In [ ]:
fig = topic_model.visualize_heatmap()
fig.write_image("../img/fig0413.png")

<img src="../img/fig0413.png"/>

In [ ]:
fig = topic_model.visualize_hierarchy()
fig.write_image("../img/fig0414.png")

<img src="../img/fig0414.png"/>

In [ ]:
fig = topic_model.visualize_documents(docs)
fig.write_image("../img/fig0415.png")

<img src="../img/fig0415.png"/>

In [ ]:
labels=topic_model.generate_topic_labels()
labels

In [ ]:
df['topic'] = ""
count = 0
for doc in docs:
    
    df.at[df.index[df['content'] == doc], 'topic'] = labels[topics[count]+1]
    count+=1

In [ ]:
df.head(20)

In [ ]:
#topic_model.reduce_topics(docs, topics, nr_topics=7)

In [ ]:
#topic_model.get_representative_docs() # entrega el doc más representativo para los 7 topicos